In [2]:
import cv2
import numpy as np
import os


from keras.models import load_model
model = load_model('D:\dipin\model_save')

In [3]:
#program original wala
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    x1, y1 = int(0.7 * frame.shape[1]), 100
    x2, y2 = frame.shape[1] - 10, int(0.45 * frame.shape[1])
    
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (100, 100)) 
    
    # Perform image processing on the ROI
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.Canny(roi, 80, 80)
    
    # Reshape the image to match the input shape expected by the model
    roi1 = roi.reshape(1, 100, 100, 1)
    
    # Predict the class index using the model
    classIndex = int(np.argmax(model.predict(roi1)))
    
    # Class labels mapping
    class_labels = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K',
                    11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T',
                    21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}
    
    cv2.putText(frame, "Alphabet : " + str(class_labels[classIndex]), (10, 430), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 2)
    
    cv2.imshow("Frame", frame)
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27:  # esc key
        break
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 73ms/step


KeyboardInterrupt: 

: 

In [4]:
#formed sentence with blue box
import cv2
import numpy as np
from keras.models import load_model
import time

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Open the camera
cap = cv2.VideoCapture(0)

# Initialize variables for forming the sentence
sentence = ""
last_recognized_time = time.time()

while True:
    # Capture the video frame-by-frame
    ret, frame = cap.read()

    # Simulating mirror image
    frame = cv2.flip(frame, 1)

    # Display the separate camera box
    cv2.rectangle(frame, (400, 100), (600, 300), (255, 0, 0), 2)

    # Create a separate camera box (ROI) for sign language input
    roi = frame[100:300, 400:600]

    # Preprocess the ROI (similar to the preprocessing done during training)
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi_canny = cv2.Canny(roi_gray, 80, 80)
    roi_resized = cv2.resize(roi_canny, (100, 100))
    roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

    # Make a prediction using the model
    class_probabilities = model.predict(roi_normalized)
    class_index = np.argmax(class_probabilities)

    # Get the predicted letter
    predicted_letter = letters_mapping[class_index]

    # If the predicted letter is not "nothing" and a certain time has passed since the last recognition
    if predicted_letter != "nothing" and time.time() - last_recognized_time > 1.5:
        if predicted_letter == 'F':  # Use 'F' as a gesture for space
            sentence += ' '
        else:
            sentence += predicted_letter
        last_recognized_time = time.time()

    # Display the formed sentence on the camera screen
    cv2.putText(frame, "Formed Sentence: " + sentence, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Sign Language Recognition', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 33ms/step


KeyboardInterrupt: 

: 

In [4]:
#box with gui to form sentence
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is not "nothing" and a certain time has passed since the last recognition
        if predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(10, self.show_video)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 66ms/step


In [5]:
#box with gui with space
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the sentence
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.sentence += ' '
            self.last_recognized_time = time.time()

        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(10, self.show_video)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 69ms/step


In [6]:
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Draw a rectangular ROI box on the video frame
        roi_box = cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Crop the frame to the defined ROI
        roi = frame[100:300, 400:600]

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is not "nothing" and a certain time has passed since the last recognition
        if predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(10, self.show_video)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 75ms/step


In [8]:
#final part 1
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the sentence
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.sentence += ' '
            self.last_recognized_time = time.time()

        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(10, self.show_video)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 83ms/step


In [2]:
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.predicted_label_var = tk.StringVar()
        self.predicted_label_var.set("Predicted Letter: ")
        self.predicted_label = tk.Label(root, textvariable=self.predicted_label_var, font=("Helvetica", 16))
        self.predicted_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the sentence
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.sentence += ' '
            self.last_recognized_time = time.time()
        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the predicted letter
        self.predicted_label_var.set("Predicted Letter: " + predicted_letter)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(100, self.show_video)  # Slow down the speed (100 milliseconds delay)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 79ms/step


In [3]:
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.predicted_letter_var = tk.StringVar()
        self.predicted_letter_var.set("Predicted Letter: ")
        self.predicted_letter_label = tk.Label(root, textvariable=self.predicted_letter_var, font=("Helvetica", 16))
        self.predicted_letter_label.pack()

        self.predicted_word_var = tk.StringVar()
        self.predicted_word_var.set("Predicted Word: ")
        self.predicted_word_label = tk.Label(root, textvariable=self.predicted_word_var, font=("Helvetica", 16))
        self.predicted_word_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""
        self.predicted_word = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the sentence
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.sentence += ' '
            self.last_recognized_time = time.time()
        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter
                self.predicted_word += predicted_letter  # Update the predicted word
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the predicted letter
        self.predicted_letter_var.set("Predicted Letter: " + predicted_letter)

        # Display the predicted word
        self.predicted_word_var.set("Predicted Word: " + self.predicted_word)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(100, self.show_video)  # Slow down the speed (100 milliseconds delay)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 57ms/step


In [9]:
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.predicted_letter_var = tk.StringVar()
        self.predicted_letter_var.set("Predicted Letter: ")
        self.predicted_letter_label = tk.Label(root, textvariable=self.predicted_letter_var, font=("Helvetica", 16))
        self.predicted_letter_label.pack()

        self.predicted_word_var = tk.StringVar()
        self.predicted_word_var.set("Predicted Word: ")
        self.predicted_word_label = tk.Label(root, textvariable=self.predicted_word_var, font=("Helvetica", 16))
        self.predicted_word_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""
        self.predicted_word = ""
        self.stage = 0  # 0: Letter, 1: Word, 2: Sentence

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def process_gesture(self, predicted_letter):
        if self.stage == 0:
            # Letter stage
            self.sentence += predicted_letter
            self.predicted_word += predicted_letter
            self.stage = 1
        elif self.stage == 1:
            # Word stage
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
                self.stage = 2
            else:
                self.sentence += predicted_letter
                self.predicted_word += predicted_letter
        elif self.stage == 2:
            # Sentence stage
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
            else:
                self.sentence += predicted_letter

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the sentence
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.sentence += ' '
            self.predicted_word += ' '
            self.stage = 0
            self.last_recognized_time = time.time()
        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.process_gesture(predicted_letter)
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the predicted letter
        self.predicted_letter_var.set("Predicted Letter: " + predicted_letter)

        # Display the predicted word
        self.predicted_word_var.set("Predicted Word: " + self.predicted_word)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(100, self.show_video)  # Slow down the speed (100 milliseconds delay)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 81ms/step


In [10]:
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.predicted_letter_var = tk.StringVar()
        self.predicted_letter_var.set("Predicted Letter: ")
        self.predicted_letter_label = tk.Label(root, textvariable=self.predicted_letter_var, font=("Helvetica", 16))
        self.predicted_letter_label.pack()

        self.predicted_word_var = tk.StringVar()
        self.predicted_word_var.set("Predicted Word: ")
        self.predicted_word_label = tk.Label(root, textvariable=self.predicted_word_var, font=("Helvetica", 16))
        self.predicted_word_label.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.sentence = ""
        self.predicted_word = ""
        self.stage = 0  # 0: Letter, 1: Word, 2: Sentence

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def process_gesture(self, predicted_letter):
        if self.stage == 0:
            # Letter stage
            self.sentence += predicted_letter
            self.predicted_word += predicted_letter
            self.stage = 1
        elif self.stage == 1:
            # Word stage
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
                self.stage = 2
            else:
                self.sentence += predicted_letter
                self.predicted_word += predicted_letter
        elif self.stage == 2:
            # Sentence stage
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.sentence += ' '
                self.reset_stages()  # Reset stages after completing a sentence
            else:
                self.sentence += predicted_letter

    def reset_stages(self):
        self.stage = 0
        self.sentence = ""
        self.predicted_word = ""

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the sentence
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.sentence += ' '
            self.predicted_word += ' '
            self.stage = 0
            self.last_recognized_time = time.time()
        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.process_gesture(predicted_letter)
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the predicted letter
        self.predicted_letter_var.set("Predicted Letter: " + predicted_letter)

        # Display the predicted word
        self.predicted_word_var.set("Predicted Word: " + self.predicted_word)

        # Display the formed sentence on the GUI
        self.sentence_var.set("Formed Sentence: " + self.sentence)

        # Repeat the video display
        self.root.after(100, self.show_video)  # Slow down the speed (100 milliseconds delay)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)
root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 78ms/step


In [2]:
#final part 2
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.letter_box_label = tk.Label(root, text="Current Letter:")
        self.letter_box_label.pack()

        self.letter_var = tk.StringVar()
        self.letter_var.set("")
        self.letter_box = tk.Label(root, textvariable=self.letter_var, font=("Helvetica", 16))
        self.letter_box.pack()

        self.word_label = tk.Label(root, text="Formed Word:")
        self.word_label.pack()

        self.word_var = tk.StringVar()
        self.word_var.set("")
        self.word_display = tk.Label(root, textvariable=self.word_var, font=("Helvetica", 16))
        self.word_display.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.letter = ""
        self.word = ""
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the letter box
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.letter = ' '
            self.last_recognized_time = time.time()

        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.letter = ' '
            else:
                self.letter = predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the current letter on the GUI
        self.letter_var.set("Current Letter: " + self.letter)

        # Repeat the video display
        self.root.after(10, self.show_video)

    def add_letter_to_word(self):
        # Add the current letter to the word
        self.word += self.letter
        self.letter = ""
        self.letter_var.set("Current Letter: " + self.letter)
        self.word_var.set("Formed Word: " + self.word)

    def add_word_to_sentence(self):
        # Add the current word to the sentence
        self.sentence += self.word + ' '
        self.word = ""
        self.word_var.set("Formed Word: " + self.word)
        self.sentence_var.set("Formed Sentence: " + self.sentence)

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)

# Add "Next" and "Enter" buttons
next_button = tk.Button(root, text="Next", command=app.add_letter_to_word)
next_button.pack()

enter_button = tk.Button(root, text="Enter", command=app.add_word_to_sentence)
enter_button.pack()

root.mainloop()

# Release the camera when the GUI is closed
app.camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 77ms/step


In [4]:
#final part 3
import cv2
import numpy as np
from keras.models import load_model
import time
import tkinter as tk
from PIL import Image, ImageTk

# Load the trained model
model = load_model('./model_save')

# Mapping index to corresponding letter or "nothing"
letters_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                   10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'nothing', 15: 'O', 16: 'P', 17: 'Q', 18: 'R',
                   19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}

# Create a class for the GUI
class SignLanguageGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Recognition")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.letter_box_label = tk.Label(root, text="Current Letter:")
        self.letter_box_label.pack()

        self.letter_var = tk.StringVar()
        self.letter_var.set("")
        self.letter_box = tk.Label(root, textvariable=self.letter_var, font=("Helvetica", 16))
        self.letter_box.pack()

        self.word_label = tk.Label(root, text="Formed Word:")
        self.word_label.pack()

        self.word_var = tk.StringVar()
        self.word_var.set("")
        self.word_display = tk.Label(root, textvariable=self.word_var, font=("Helvetica", 16))
        self.word_display.pack()

        self.sentence_label = tk.Label(root, text="Formed Sentence:")
        self.sentence_label.pack()

        self.sentence_var = tk.StringVar()
        self.sentence_var.set("")
        self.sentence_display = tk.Label(root, textvariable=self.sentence_var, font=("Helvetica", 16))
        self.sentence_display.pack()

        self.last_recognized_time = time.time()
        self.letter = ""
        self.word = ""
        self.sentence = ""

        self.camera = cv2.VideoCapture(0)
        self.show_video()

    def show_video(self):
        _, frame = self.camera.read()
        frame = cv2.flip(frame, 1)

        # Create a separate camera box (ROI) for sign language input
        roi = frame[100:300, 400:600]

        # Draw a rectangle (visible square) around the ROI
        cv2.rectangle(frame, (400, 100), (600, 300), (0, 255, 0), 2)

        # Preprocess the ROI (similar to the preprocessing done during training)
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_canny = cv2.Canny(roi_gray, 80, 80)
        roi_resized = cv2.resize(roi_canny, (100, 100))
        roi_normalized = roi_resized.reshape(1, 100, 100, 1) / 255.0

        # Make a prediction using the model
        class_probabilities = model.predict(roi_normalized)
        class_index = np.argmax(class_probabilities)

        # Get the predicted letter
        predicted_letter = letters_mapping[class_index]

        # If the predicted letter is 'nothing', add a space to the letter box
        if predicted_letter == "nothing" and time.time() - self.last_recognized_time > 1.5:
            self.letter = ' '
            self.last_recognized_time = time.time()

        # If the predicted letter is not 'nothing' and a certain time has passed since the last recognition
        elif predicted_letter != "nothing" and time.time() - self.last_recognized_time > 1.5:
            if predicted_letter == 'F':  # Use 'F' as a gesture for space
                self.letter = ' '
            else:
                self.letter = predicted_letter
            self.last_recognized_time = time.time()

        # Display the video frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)
        self.video_label.img = img
        self.video_label.config(image=img)

        # Display the current letter on the GUI
        self.letter_var.set("Current Letter: " + self.letter)

        # Repeat the video display
        self.root.after(10, self.show_video)

    def add_letter_to_word(self):
        # Add the current letter to the word
        self.word += self.letter
        self.letter = ""
        self.letter_var.set("Current Letter: " + self.letter)
        self.word_var.set("Formed Word: " + self.word)

    def add_word_to_sentence(self):
        # Add the current word to the sentence
        self.sentence += self.word + ' '
        self.word = ""
        self.word_var.set("Formed Word: " + self.word)
        self.sentence_var.set("Formed Sentence: " + self.sentence)

    def close_program(self):
        # Release the camera when the GUI is closed
        self.camera.release()
        cv2.destroyAllWindows()
        self.root.destroy()

# Create the main window and run the GUI
root = tk.Tk()
app = SignLanguageGUI(root)

# Add "Next," "Enter," and "Close" buttons
next_button = tk.Button(root, text="Next", command=app.add_letter_to_word)
next_button.pack()

enter_button = tk.Button(root, text="Enter", command=app.add_word_to_sentence)
enter_button.pack()

close_button = tk.Button(root, text="Close", command=app.close_program)
close_button.pack()

root.mainloop()


1/1 [==============================] - 0s 34ms/step
